In [ ]:
import torch
import torch.nn as nn
import torchvision.models as model, torchvision
import torchvision.transforms as transforms
from PIL import Image

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)

100%|██████████| 170498071/170498071 [00:17<00:00, 9610228.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        self.
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.max_pool2d(x, (2, 2))
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
net = LeNet()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        net.train()
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 2.235
[1,  4000] loss: 1.834
[1,  6000] loss: 1.661
[1,  8000] loss: 1.578
[1, 10000] loss: 1.499
[1, 12000] loss: 1.473
[2,  2000] loss: 1.375
[2,  4000] loss: 1.363
[2,  6000] loss: 1.338
[2,  8000] loss: 1.309
[2, 10000] loss: 1.313
[2, 12000] loss: 1.284
[3,  2000] loss: 1.207
[3,  4000] loss: 1.215
[3,  6000] loss: 1.189
[3,  8000] loss: 1.198
[3, 10000] loss: 1.198
[3, 12000] loss: 1.171
[4,  2000] loss: 1.086
[4,  4000] loss: 1.109
[4,  6000] loss: 1.093
[4,  8000] loss: 1.109
[4, 10000] loss: 1.103
[4, 12000] loss: 1.112
[5,  2000] loss: 1.022
[5,  4000] loss: 1.030
[5,  6000] loss: 1.044
[5,  8000] loss: 1.031
[5, 10000] loss: 1.043
[5, 12000] loss: 1.038


In [ ]:
torch.save(net.state_dict(), 'lenet.pth')

In [ ]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

Files already downloaded and verified


In [ ]:
net = LeNet()

net.load_state_dict(torch.load('lenet.pth'))

<All keys matched successfully>

In [ ]:
def test_network(net, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

In [ ]:
test_network(net, test_loader)

Accuracy of the network on the 10000 test images: 59 %


In [ ]:
image = Image.open('/content/download.jpg')
image = transform(image).unsqueeze(0)

# Test the image
output = net(image)
_, predicted = torch.max(output, 1)
print('Predicted class:', predicted.item())

FileNotFoundError: [Errno 2] No such file or directory: '/content/download.jpg'

#Part 2

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)


In [ ]:
net = AlexNet()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        # print('1')
        optimizer.step()

        # print('2')
        running_loss += loss.item()
        if i % 30 == 29:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        # print('3')
